In [1]:
from selenium import webdriver
from urllib.request import urlopen
from urllib.parse import quote_plus
import time
import re
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup

In [2]:
driver=webdriver.Chrome("C:/mychrome/chromedriver.exe")

In [44]:
#방탄소년단
url="https://www.melon.com/artist/song.htm?artistId=672375"
driver.get(url)

In [45]:
#곡 정보 리스트 
songdata=[]

In [46]:
#가수별 크롤링
def nextcontent(driver):
    for n in range(1,8): #가수별 곡 페이지 수에 따라 바뀜
        for i in range(1,51):
            right=driver.find_element_by_css_selector("tbody > tr:nth-child(%d) > td:nth-child(3) > div > div > a.btn.btn_icon_detail > span" %i)
            right.click()
            time.sleep(1)
            html=driver.page_source
            soup=BeautifulSoup(html,"lxml")
            genre=soup.select(" div > div.entry > div.meta > dl > dd:nth-of-type(3)")[0].text
            if genre=='J-POP':
                pass
            else:
                #가사 펼치기
                try:
                    right=driver.find_element_by_css_selector("button.button_more.arrow_d")
                    right.click()
                    time.sleep(1)
                except:
                    pass
                # 곡 정보 가져오기
                def get_content(driver):
                    html=driver.page_source
                    soup=BeautifulSoup(html,"lxml")#html.parser와 같음 조금가볍다함
                    try:
                        singer=soup.select("div.artist > a > span")[0].text 
                    except:
                        singer="Null"
                    try:
                        song=soup.select("div.song_name")[0].text.replace("\t","").replace("\n","").replace("곡명","")
                    except:
                        song="Null"
                    try:
                        temp=soup.select("div.lyric.on ")
                        lyrics=str(temp).replace("<br/>","\n").replace('<div class="lyric on" id="d_video_summary" tabindex="0"><!-- height:auto; 로 변경시, 확장됨 -->\n\t\t\t\t\t\t\t',"").replace("</div>","")
                    except:
                        lyrics="Null"
                    try:
                        lyricist=soup.select("div.ellipsis.artist")[0].text.replace("\n","")
                    except:
                        lyricist="Null"
                    data=[singer,song,lyrics,lyricist] #리스트구조
                    songdata.append(data)
            get_content(driver)
            driver.back()
        if n%10!=0:
            if n<10:
                a=n+1
            else:
                t=n%10
                a=t+1
            right=driver.find_element_by_css_selector("div > span.page_num > a:nth-child(%d)"%a)
            right.click()
            time.sleep(1)
        else:
            right=driver.find_element_by_css_selector("div.paging > a.next")
            right.click()
            time.sleep(1)

nextcontent(driver)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"tbody > tr:nth-child(21) > td:nth-child(3) > div > div > a.btn.btn_icon_detail > span"}
  (Session info: chrome=88.0.4324.190)


In [47]:
# 데이터프레임으로 바꾸기
song=pd.DataFrame(songdata,columns=['singer','song','lyrics','lyricist'])

In [48]:
song

,singer,song,lyrics,lyricist
0,방탄소년단,Dynamite (Holiday Remix),[Cos ah ah I’m in the stars tonight\n\nShoes o...,David Stewart
1,방탄소년단,Life Goes On,[어느 날 세상이 멈췄어\n아무런 예고도 하나 없이\n봄은 기다림을 몰라서\n눈치 ...,Pdogg
2,방탄소년단,내 방을 여행하는 법,[떠나볼까 let me fly to my\n시선을 낮추고 어디든 막 zoom\n지금...,Cosmo's Midnight
3,방탄소년단,Blue & Grey,[Where is my angel\n하루의 끝을 드리운\nSomeone come a...,Jisoo Park (153/Joombas)
4,방탄소년단,Skit,[],RM
...,...,...,...,...
315,방탄소년단,Dream Glow,[],Tor Erik Hermansen
316,방탄소년단,A Brand New Day,[],Alexander George Edward Crossan
317,방탄소년단,All Night,[],Powers Pleasant
318,방탄소년단,Skit : 망설임과 두려움,[],방시혁


In [50]:
# 텍스트파일로 저장
song.to_csv("방탄소년단.txt",encoding="utf-8")